<a href="https://colab.research.google.com/github/SeydouBamba/Test/blob/main/Copie_de_Analyse_et_visualisation_de_donn%C3%A9es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DEVOIR DE BIG DATA INGC2-ESMT_2023

Les membres du Groupe 

Seydou BAMBA

Lassana DEMBE

Galyam Boubacar Georges  OUEDRAOGO

# Introduction
Le but principal de ce notebook est de maîtriser <b>Spark SQL</b> et de pouvoir défnir des fonctions de fenêtre Spark et des fonctions Python enregistrées dans le registre Spark afin d'être utilisées dans les requêtes SQL.

Le même jeu de données du précédent TP sera utilisé avec le même environnement d'exécution.


# Initialisation de l'environnement d'exécution

Installation du JDK

*   Élément de liste
*   Élément de liste



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Téléchargement de l'archive du framework Apache Spark

In [2]:
# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

Extraction de l'archive dans le dossier courant <mark>/content</mark>

In [3]:
# Unzip the file
!tar xf spark-3.3.2-bin-hadoop3.tgz

Installation des modules Python <b>pyspark</b> et <b>findspark</b>

In [4]:
!pip install -q pyspark
!pip install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.8 MB/s eta 0:00:00


Test de l'installation de pyspark

In [5]:
!find /content -name "pyspark"

/content/spark-3.3.2-bin-hadoop3/bin/pyspark
/content/spark-3.3.2-bin-hadoop3/python/pyspark
/content/spark-3.3.2-bin-hadoop3/python/pyspark/python/pyspark


Création des variables d'environnement <mark>SPARK_HOME</mark> et <mark>JAVA_HOME</mark> pour situer respectivement les emplacements d'installation de Spark et Java 

In [6]:
import os
os.environ["SPARK_HOME"] =  "/content/spark-3.3.2-bin-hadoop3" 
os.environ["JAVA_HOME"] ="/usr/lib/jvm/java-8-openjdk-amd64"

Importation des bibliothèques Spark SQL

In [7]:
import findspark 
print("findspark.init() initialise les variables d'environnement pour spark") 
findspark.init() 

# Pyspark session objects
from pyspark.sql import SparkSession 
# Pyspark session configuration
from pyspark import SparkConf  

# Pyspark functions
import pyspark.sql.functions as f
from pyspark.sql import * 

# Pyspark SQL data types
from pyspark.sql.types import *

findspark.init() initialise les variables d'environnement pour spark


# Analyse et visualisation de données

## Définition de fonctions pour l'environnement PySpark

La fonction <mark>demarrer_spark</mark> permet d'initialiser une session <i>client</i> avec Spark

In [8]:
def demarrer_spark():
  local = "local[*]"
  appName = "TP3"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "100G").\
  set("spark.driver.memory","50G").\
  set("spark.sql.catalogImplementation","in-memory").\
  set("spark.driver.maxResultSize", "10G")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  # spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")
  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  # spark.conf.set("spark.sql.shuffle.partitions","200")    

  print("session démarrée, son id est ", sc.applicationId)
  return spark

Démarrage de la session

In [9]:
spark = demarrer_spark()

session démarrée, son id est  local-1677177577013


En vue de simplifier l'exécution des requêtes SQL, nous définissons la commande magique &#128526; <b><font color="blue">%%sql</font></b> pour exécuter les requêtes plus facilement

In [10]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)
import gc

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    resultat = None
    est_une_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          resultat = spark.sql(r)
          est_une_requete = r.lower().startswith('select') or r.lower().startswith('with')  
    if(est_une_requete):
      # Explain the execution plan
      resultat.explain()
      # Display the result
      return display(resultat)
    else:
      return print('ok')

De même, nous redéfinissons la fonction <b>display</b> pour un meilleur affichage des données manipulées.

In [11]:
import pandas as pd

def display(df, n=10):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  pdf = df.limit(n).toPandas()
  # Free memory
  df.unpersist()
  # Force Spark to free memory
  spark.catalog.clearCache()
  # and Python too
  gc.collect(2)
  return pdf

print("display redéfini")

display redéfini


## Définition de fonctions de visualisation

Fonction d'exécution de requête SQL et conversion du résultat (un Dataframe Spark) en Dataframe Pandas

In [12]:
import gc

def getPandasDataFrame(sqlQuery):
  # Execute SQL Query with PySpark
  dfSpark = spark.sql(sqlQuery)
  # Convert Spark dataframe to Pandas dataframe
  pdf = dfSpark.toPandas()
  # Force Spark to free memory
  dfSpark.unpersist()
  spark.catalog.clearCache()
  # and Python too
  gc.collect(2)
  # Return the Pandas Dataframe
  return pdf

Fonctions de visualisation

In [13]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.tools as pt
import numpy as np
import math

def drawLine(sql):
  # Getting Pandas Dataframe
  pdf = getPandasDataFrame(sql)
  # plotting the line chart
  fig = px.line(pdf, x=pdf.columns[0], y=pdf.columns[1])
  # showing the plot
  fig.show()

def drawBar(sql):
  # Getting Pandas Dataframe
  pdf = getPandasDataFrame(sql)
  # plotting the bar chart
  fig = px.bar(pdf, x=pdf.columns[0], y=pdf.columns[1])
  # showing the plot
  fig.show()

def drawHistogram(sql):
  # Getting Pandas Dataframe
  pdf = getPandasDataFrame(sql)
  # plotting the histogram chart
  fig = px.histogram(pdf, x=pdf.columns[0], y=pdf.columns[1])
  # showing the plot
  fig.show()

def drawHeatmap(sql, scale=lambda x: x):
  # Getting Pandas Dataframe
  pdf = getPandasDataFrame(sql)
  if len(pdf.columns) != 3 and not (pdf[pdf.columns[2]].dtype == np.float64 or pdf[pdf.columns[2]].dtype == np.int64):
    raise Exception("Sorry, no numbers below zero")
  source = pdf[pdf.columns[0]].tolist()
  target = pdf[pdf.columns[1]].tolist()
  value = pdf[pdf.columns[2]].tolist()
  # plotting the figure
  fig = go.Figure(data = go.Heatmap(x = source, y = target, z = [scale(x) for x in value])) 
  fig.show()

def drawPie(sql):
  # Getting Pandas Dataframe
  pdf = getPandasDataFrame(sql)
  # plotting the pie chart
  fig = px.pie(pdf, names=pdf.columns[0], values=pdf.columns[1])
  # showing the plot
  fig.show()

def drawStackedBar(sql):
  # Getting Pandas Dataframe
  pdf = getPandasDataFrame(sql)
  # plotting the stacked bar chart
  fig = px.bar(df, x=pdf.columns[0], y=pdf.columns[2], color=pdf.columns[1], hover_data=pdf.columns[1], barmode = 'stack')
  # showing the plot
  fig.show()

def drawSankey(sql):
  # Getting Pandas Dataframe
  pdf = getPandasDataFrame(sql)
  
  labels = []
  x = set(pdf[pdf.columns[0]].tolist())
  dicX = {}
  i = 0
  for e in x:
    dicX[e] = i
    labels.append(e)
    i += 1
    
  y = set(pdf[pdf.columns[1]].tolist())
  dicY = {}
  # i = len(labels)
  for e in y:
    if(e in dicX):
      dicY[e] = dicX[e]
    else:
      dicY[e] = i
      i += 1
    labels.append(e)

  fig = go.Figure(data=[go.Sankey(
    node = dict(
      thickness = 5,
      line = dict(color = "green", width = 0.1),
      label = labels,
      color = "blue"
    ),
    link = dict(
      # indices correspond to labels
      source = [dicX[e] for e in pdf[pdf.columns[0]].tolist()],
      target = [dicY[e] for e in pdf[pdf.columns[1]].tolist()],
      value = pdf[pdf.columns[2]].tolist()
  ))])

  # showing the plot
  fig.show()

## Récupération du jeu de données

Téléchargement du jeu

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!unzip -o /content/drive/MyDrive/ecommerce-behavior-data-from-multi-category-store.zip

Archive:  /content/drive/MyDrive/ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            


In [16]:
#!curl -L -o ecommerce-behavior-data-from-multi-category-store.zip 'https://drive.google.com/u/0/uc?id=169buWYGmSpayzJO7dtsy61HHvPXd4OI1&export=download&confirm=t'
#!curl -L -o ecommerce-behavior-data-from-multi-category-store.zip 'https://drive.google.com/u/0/uc?id=1CVhmxsU3GY0FYGS1uP3m_tGbyGjEfuQc&export=download&confirm=t'

Extraction des données

In [17]:
!unzip -o ecommerce-behavior-data-from-multi-category-store.zip
!ls .

unzip:  cannot find or open ecommerce-behavior-data-from-multi-category-store.zip, ecommerce-behavior-data-from-multi-category-store.zip.zip or ecommerce-behavior-data-from-multi-category-store.zip.ZIP.
2019-Nov.csv  drive	   spark-3.3.2-bin-hadoop3
2019-Oct.csv  sample_data  spark-3.3.2-bin-hadoop3.tgz


Aperçu de format des données

In [18]:
!head -10 2019-Nov.csv

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
2019-11-01 00:00:01 UTC,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
2019-11-01 00:00:01 UTC,view,1306421,2053013558920217191,computers.notebook,hp,514.56,51402

Chargement du jeu de données dans Spark

In [19]:
df = spark.read.csv("2019-Nov.csv", header=True, sep=',')
display(df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,None,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
5,2019-11-01 00:00:01 UTC,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
6,2019-11-01 00:00:01 UTC,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
7,2019-11-01 00:00:02 UTC,view,15900065,2053013558190408249,None,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
8,2019-11-01 00:00:02 UTC,view,12708937,2053013553559896355,None,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24
9,2019-11-01 00:00:02 UTC,view,1004258,2053013555631882655,electronics.smartphone,apple,732.07,532647354,d2d3d2c6-631d-489e-9fb5-06f340b85be0


Affichage du nombre d'enregistrements du jeu de données

In [20]:
#df.count()

Affichage du schéma du Dataframe Spark

In [21]:
df.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



Casting de certaines colonnes aux types de données attendus

In [22]:
df = df.withColumn("event_time",df.event_time.cast(TimestampType()))
df = df.withColumn("product_id",df.product_id.cast(IntegerType()))
df = df.withColumn("category_id",df.category_id.cast(IntegerType()))
df = df.withColumn("price",df.price.cast(DoubleType()))
df = df.withColumn("user_id",df.user_id.cast(IntegerType()))

Affichage du nouveau schéma du Dataframe Spark

In [23]:
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



Matérialisation du dataframe comme une vue SQL avec la vue <mark>events</mark> qui pointe sur lui

In [24]:
df.createOrReplaceTempView('events')

## Activité 1

Test de notre première requête SQL. <b>Que fait-elle ?</b>

In [ ]:
%%sql
SELECT brand, event_type, COUNT(*) AS nb FROM events GROUP BY brand, event_type

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[brand#22, event_type#18], functions=[count(1)])
   +- Exchange hashpartitioning(brand#22, event_type#18, 200), ENSURE_REQUIREMENTS, [plan_id=87]
      +- HashAggregate(keys=[brand#22, event_type#18], functions=[partial_count(1)])
         +- FileScan csv [event_type#18,brand#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/2019-Nov.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<event_type:string,brand:string>




ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

Que fait cette deuxième requête ?

In [ ]:
drawLine("SELECT brand, COUNT(*) AS nb FROM events GROUP BY brand")

Voici la liste des types d'évènement possibles : <i>purchase, view, cart</i> ...

In [ ]:
%%sql
SELECT DISTINCT event_type FROM events

Et leur répartition dans le jeu de données

In [ ]:
drawPie("SELECT event_type, COUNT(*) FROM events GROUP BY event_type")

Comment comprenez-vous cette requête et sa visualisation ?

In [ ]:
sql = "SELECT brand, event_type, COUNT(*) AS nb FROM events GROUP BY brand, event_type"
drawHeatmap(sql, scale=math.log)

Que fait cette dernière requête ?
Comment comprenez-vous sa graphique ?

In [ ]:
sql = "SELECT category_code, date_format(event_time, 'L'), SUM(price) AS amount FROM events WHERE event_type='purchase' AND category_code LIKE 'electronics.%' GROUP BY category_code, date_format(event_time, 'L')"
drawSankey(sql)

In [ ]:
sql = "SELECT event_type as A, category_code as B, COUNT(*) FROM events WHERE event_type='cart' GROUP BY event_type, category_code \
 UNION SELECT category_code as A, event_type as B, COUNT(*) FROM events WHERE event_type='purchase' GROUP BY event_type, category_code"
drawSankey(sql)

In [ ]:
spark.catalog.clearCache()
gc.collect(2)

## Activité 2
Nous continuons à travailler avec la même vue <mark>events</mark>. Le but est d’écrire vos propres requêtes <u>et de réflechir sur leurs meilleures présentations</u>.
<ol>
<li>Quel est le chiffre d'affaire réalisé selon les jours de la semaine ?</li>
<li>Donner l'évolution du nombre de vues de produits selon les jours du mois.</li>
<li>Donner le top 3 des catégories de produits par type d'évènement.</li>
<li>Donner le chiffre d'affaire gagné sur chaque marque selon le jour de la semaine.</li>
<li>Donner le top 3 des marques pour catégorie de produit.</li>
</ol>

In [ ]:
# 1. Quel est le chiffre d'affaire réalisé selon les jours de la semaine ? utiliser pour comparaisons
%%sql
SELECT date_format(event_time, 'E'), SUM(price) AS ChiffreDAffaire FROM events WHERE event_type='purchase'  GROUP BY date_format(event_time, 'E')

,"date_format(event_time, E)",ChiffreDAffaire
0,Fri,31317619.23
1,Sat,50923665.08
2,Sun,77535628.20
3,Mon,30593864.09
4,Tue,28217036.28
5,Wed,28461191.93
6,Thu,28145885.69


In [ ]:
# visualisation
drawHistogram(
"SELECT date_format(event_time, 'E'), SUM(price) AS ChiffreDAffaire FROM events WHERE event_type='purchase'  GROUP BY date_format(event_time, 'E')")

In [ ]:
# 2. Donner l'évolution du nombre de vues de produits selon les jours du mois.
%%sql
SELECT date_format(event_time, 'd') AS jour, COUNT(*) FROM events WHERE event_type='view' GROUP BY date_format(event_time, 'L'), date_format(event_time, 'd')

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[_groupingexpression#100, _groupingexpression#101], functions=[count(1)])
   +- Exchange hashpartitioning(_groupingexpression#100, _groupingexpression#101, 200), ENSURE_REQUIREMENTS, [plan_id=47]
      +- HashAggregate(keys=[_groupingexpression#100, _groupingexpression#101], functions=[partial_count(1)])
         +- Project [date_format(event_time#44, L, Some(Etc/UTC)) AS _groupingexpression#100, date_format(event_time#44, d, Some(Etc/UTC)) AS _groupingexpression#101]
            +- Project [cast(event_time#17 as timestamp) AS event_time#44]
               +- Filter (isnotnull(event_type#18) AND (event_type#18 = view))
                  +- FileScan csv [event_time#17,event_type#18] Batched: false, DataFilters: [isnotnull(event_type#18), (event_type#18 = view)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/2019-Nov.csv], PartitionFilters: [], PushedFilters: [IsNotNull(event_type), EqualTo(ev

,jour,count(1)
0,1,1403991
1,2,1514324
2,3,1525418
3,4,1744279
4,5,1673138
5,6,1649832
6,7,1752524
7,8,1793319
8,9,1785006
9,10,1846107


In [ ]:
# Visualisation
drawLine("SELECT date_format(event_time, 'd') AS jour, COUNT(*) AS nombreVue FROM events WHERE event_type='view' GROUP BY date_format(event_time, 'L'), date_format(event_time, 'd')")

In [ ]:
# 3. Donner le top 3 des catégories de produits par type d'évènement.
%%sql 
WITH chiffreDaffaire_par_Categorie AS (SELECT event_type, category_code, SUM(price) AS total FROM events GROUP BY event_type, category_code), 
ca_et_classement_par_Categorie AS (SELECT event_type, category_code, total, RANK() over (PARTITION BY event_type ORDER BY total DESC) AS classement FROM chiffreDaffaire_par_Categorie) SELECT  event_type, category_code, total, classement FROM ca_et_classement_par_Categorie WHERE classement <= 3;


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Filter (classement#96 <= 3)
   +- Window [rank(total#95) windowspecdefinition(event_type#18, total#95 DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS classement#96], [event_type#18], [total#95 DESC NULLS LAST]
      +- Sort [event_type#18 ASC NULLS FIRST, total#95 DESC NULLS LAST], false, 0
         +- Exchange hashpartitioning(event_type#18, 200), ENSURE_REQUIREMENTS, [plan_id=52]
            +- HashAggregate(keys=[event_type#18, category_code#21], functions=[sum(price#75)])
               +- Exchange hashpartitioning(event_type#18, category_code#21, 200), ENSURE_REQUIREMENTS, [plan_id=49]
                  +- HashAggregate(keys=[event_type#18, category_code#21], functions=[partial_sum(price#75)])
                     +- Project [event_type#18, category_code#21, cast(price#23 as double) AS price#75]
                        +- FileScan csv [event_type#18,category_code#21,price#23] Batch

,event_type,category_code,total,classement
0,cart,electronics.smartphone,5.363704e+08,1
1,cart,None,1.030350e+08,2
2,cart,electronics.video.tv,4.446716e+07,3
3,purchase,electronics.smartphone,1.778217e+08,1
4,purchase,None,2.988051e+07,2
5,purchase,electronics.video.tv,1.245715e+07,3
6,view,electronics.smartphone,7.205377e+09,1
7,view,None,3.735317e+09,2
8,view,computers.notebook,1.531980e+09,3


In [ ]:
# 4. Donner le chiffre d'affaire gagné sur chaque marque.
%%sql
SELECT brand, SUM(price) AS ChiffreDAffaire FROM events WHERE event_type='purchase' GROUP BY brand ORDER BY ChiffreDAffaire DESC

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [ChiffreDAffaire#119 DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(ChiffreDAffaire#119 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=204]
      +- HashAggregate(keys=[brand#22], functions=[sum(price#75)])
         +- Exchange hashpartitioning(brand#22, 200), ENSURE_REQUIREMENTS, [plan_id=201]
            +- HashAggregate(keys=[brand#22], functions=[partial_sum(price#75)])
               +- Project [brand#22, cast(price#23 as double) AS price#75]
                  +- Filter (isnotnull(event_type#18) AND (event_type#18 = purchase))
                     +- FileScan csv [event_type#18,brand#22,price#23] Batched: false, DataFilters: [isnotnull(event_type#18), (event_type#18 = purchase)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/2019-Nov.csv], PartitionFilters: [], PushedFilters: [IsNotNull(event_type), EqualTo(event_type,purchase)], ReadSchema: struct<event_type:string,brand:string,pri

,brand,ChiffreDAffaire
0,apple,1.275125e+08
1,samsung,5.486988e+07
2,xiaomi,1.125987e+07
3,None,1.102572e+07
4,lg,5.239019e+06
5,huawei,4.780682e+06
6,sony,3.862886e+06
7,lucente,3.527546e+06
8,oppo,3.488541e+06
9,acer,3.347307e+06


In [ ]:
# 5. Donner le top 3 des marques pour catégorie de produit.
%%sql 
WITH chiffreDaffaire_par_Marque AS (SELECT category_code, brand, SUM(price) AS total FROM events GROUP BY category_code, brand), 
ca_et_classement_par_Marque AS (SELECT category_code, brand, total, RANK() over (PARTITION BY category_code ORDER BY total DESC) AS classement FROM chiffreDaffaire_par_Marque) SELECT category_code, brand, total, classement FROM ca_et_classement_par_Marque WHERE classement <= 3;


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Filter (classement#131 <= 3)
   +- Window [rank(total#130) windowspecdefinition(category_code#21, total#130 DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS classement#131], [category_code#21], [total#130 DESC NULLS LAST]
      +- Sort [category_code#21 ASC NULLS FIRST, total#130 DESC NULLS LAST], false, 0
         +- Exchange hashpartitioning(category_code#21, 200), ENSURE_REQUIREMENTS, [plan_id=289]
            +- HashAggregate(keys=[category_code#21, brand#22], functions=[sum(price#75)])
               +- Exchange hashpartitioning(category_code#21, brand#22, 200), ENSURE_REQUIREMENTS, [plan_id=286]
                  +- HashAggregate(keys=[category_code#21, brand#22], functions=[partial_sum(price#75)])
                     +- Project [category_code#21, brand#22, cast(price#23 as double) AS price#75]
                        +- FileScan csv [category_code#21,brand#22,price#23] Batched: f

,category_code,brand,total,classement
0,None,None,1.227760e+09,1
1,None,lucente,2.864295e+08,2
2,None,sony,1.797310e+08,3
3,accessories.bag,karya,3.887804e+06,1
4,accessories.bag,None,3.328868e+06,2
5,accessories.bag,coccinelle,1.827197e+06,3
6,accessories.umbrella,fabretti,6.812736e+04,1
7,accessories.umbrella,xiaomi,1.834560e+04,2
8,accessories.umbrella,hoco,1.113243e+04,3
9,accessories.wallet,petek,1.551865e+06,1


In [ ]:
# Visualisation
drawSankey(
    "WITH chiffreDaffaire_par_Marque AS (SELECT category_code, brand, SUM(price) AS total FROM events GROUP BY category_code, brand),ca_et_classement_par_Marque AS (SELECT category_code, brand, total, RANK() over (PARTITION BY category_code ORDER BY total DESC) AS classement FROM chiffreDaffaire_par_Marque) SELECT category_code, brand, total, classement FROM ca_et_classement_par_Marque WHERE classement <= 3;")

6.A quel jour de la semaine, a-t-on le plus d'activités sur le site ? L'activité correspond au évènements de la journée

In [ ]:
%%sql
SELECT date_format(event_time, 'E') AS jour, COUNT(*) AS nombre FROM events GROUP BY date_format(event_time, 'E') ORDER BY 2 DESC
    LIMIT 1;

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=1, orderBy=[nombre#96L DESC NULLS LAST], output=[jour#95,nombre#96L])
   +- HashAggregate(keys=[_groupingexpression#100], functions=[count(1)])
      +- Exchange hashpartitioning(_groupingexpression#100, 200), ENSURE_REQUIREMENTS, [plan_id=44]
         +- HashAggregate(keys=[_groupingexpression#100], functions=[partial_count(1)])
            +- Project [date_format(cast(event_time#17 as timestamp), E, Some(Etc/UTC)) AS _groupingexpression#100]
               +- FileScan csv [event_time#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/2019-Nov.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<event_time:string>




,jour,nombre
0,Sat,13252995


In [ ]:
# Visualisation
drawHistogram("SELECT date_format(event_time, 'E') AS jour, COUNT(*) AS activity FROM events GROUP BY date_format(event_time, 'E')")

7.Dessinez l'histogramme de l'évolution journalière des activités du mois de Novembre

In [ ]:
drawHistogram("SELECT date_format(event_time, 'd') AS jour, COUNT(*) AS evolution FROM events GROUP BY date_format(event_time, 'L'), date_format(event_time, 'd')")

8.Quelle est la valeur moyenne des commandes des utilisateurs ? Les produits achetés durant la même session font partie d'une même commande.

In [ ]:
# 8.Quelle est la valeur moyenne des commandes des utilisateurs ? Les produits achetés durant la même session font partie d'une même commande.
%%sql
SELECT user_session,  AVG(price) AS valeurMoyenneCommandes  FROM events  WHERE event_type = 'purchase' GROUP BY user_session;

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[user_session#25], functions=[avg(price#75)])
   +- Exchange hashpartitioning(user_session#25, 200), ENSURE_REQUIREMENTS, [plan_id=490]
      +- HashAggregate(keys=[user_session#25], functions=[partial_avg(price#75)])
         +- Project [cast(price#23 as double) AS price#75, user_session#25]
            +- Filter (isnotnull(event_type#18) AND (event_type#18 = purchase))
               +- FileScan csv [event_type#18,price#23,user_session#25] Batched: false, DataFilters: [isnotnull(event_type#18), (event_type#18 = purchase)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/2019-Nov.csv], PartitionFilters: [], PushedFilters: [IsNotNull(event_type), EqualTo(event_type,purchase)], ReadSchema: struct<event_type:string,price:string,user_session:string>




,user_session,valeurMoyenneCommandes
0,8693715b-2f32-462b-b40b-5535eb559b25,66.90
1,ad3086a0-22ad-4da4-8c2e-695bdf3fb085,185.85
2,5b0a2af3-293f-4bd2-b516-8ec16ca6162c,206.96
3,3e5e22a7-ffd8-4023-8b28-fe71da370eab,230.93
4,279f59b0-d73f-445d-84db-29a984780005,458.28
5,d6b4aff1-968c-4789-b1a8-edee55fb1e89,128.42
6,a8c5898f-bb2f-45dd-90dd-52e1576c55e2,94.98
7,c662b261-1af9-41fa-b3ce-dedbf2b553de,33.44
8,d6fd85b7-6b95-4447-8526-8516d1bddb0d,308.63
9,04934e84-18dc-4914-9d87-c17cebf28a85,178.87


9.Donnez le top 3 des produits dont les taux de conversion sont les plus faibles (Taux de conversion = Nombre de vues / Nombre d'achats).

In [30]:
%%sql
SELECT brand, SUM(if(event_type='purchase', 1, 0)) AS nbreAchat, SUM(if(event_type='view', 1, 0)) AS nbreVues, 
(SUM(if(event_type='purchase', 1, 0)) / SUM(if(event_type='view', 1, 0))) AS tauxDeConversion 
FROM events GROUP BY brand ORDER BY tauxDeConversion LIMIT 3;


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=3, orderBy=[tauxDeConversion#164 ASC NULLS FIRST], output=[brand#22,nbreAchat#162L,nbreVues#163L,tauxDeConversion#164])
   +- HashAggregate(keys=[brand#22], functions=[sum(if ((event_type#18 = purchase)) 1 else 0), sum(if ((event_type#18 = view)) 1 else 0)])
      +- Exchange hashpartitioning(brand#22, 200), ENSURE_REQUIREMENTS, [plan_id=361]
         +- HashAggregate(keys=[brand#22], functions=[partial_sum(if ((event_type#18 = purchase)) 1 else 0), partial_sum(if ((event_type#18 = view)) 1 else 0)])
            +- FileScan csv [event_type#18,brand#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/2019-Nov.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<event_type:string,brand:string>




,brand,nbreAchat,nbreVues,tauxDeConversion
0,tec,0,2118,0.0
1,wengemotion,0,189,0.0
2,stikbot,0,204,0.0


In [30]:
# Visualisation
drawLine("SELECT brand,(SUM(if(event_type='purchase', 1, 0)) / SUM(if(event_type='view', 1, 0))) AS tauxDeConversion FROM events GROUP BY product_id, brand ORDER BY tauxDeConversion")

10.Dessinez le diagramme de Sankey des catégories de produit que les utilisateurs observent souvent durant une même session.
